<a href="https://colab.research.google.com/github/KevinFallon/mnist_project/blob/main/MNIST_ResNet_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn as nn
import torchvision as tv
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

In [5]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [6]:
# HYPER PARAMETERS
num_epochs = 5
learning_rate = 0.1
batch_size = 128

In [7]:
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=tv.transforms.ToTensor())
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=tv.transforms.ToTensor())

minst_train_loader = DataLoader(mnist_trainset, batch_size=batch_size, shuffle=True)
mnist_test_loader = DataLoader(mnist_testset, batch_size=batch_size, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 350704866.72it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 21685587.87it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 121286743.42it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3970514.54it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [8]:
from torchvision.models.googlenet import GoogLeNet, BasicConv2d

class MNISTGoogLeNet(GoogLeNet):
  def __init__(self):
    # Num classes = 10 because we're predicting 0-9.
    super(MNISTGoogLeNet, self).__init__()
    # Keep same parameters except we change the number of input channels.
    self.conv1 = BasicConv2d(1, 64, kernel_size=7, stride=2, padding=3)
    # .. rest of layers
    self.fc = nn.Linear(1024, 10) # output is 10 because we recognize 10 classes

In [9]:
def train(model, optimizer, data_loader, loss_module, epochs, batch_size):
  # Set model to train mode
  model.train()

  total_step = len(data_loader)
  # Training loop
  for epoch in range(epochs):
    for i, (data_inputs, data_labels) in enumerate(data_loader):

      # Step 1: Move the input data to the device (GPU)
      data_inputs = data_inputs.to(device)
      data_labels = data_labels.to(device)

      # Step 2: Run the model on the input data
      # Pluck the first item because GoogLeNet's forward function outputs a
      # tuple of 3 values. The first value is the tensor we care about.
      preds = model(data_inputs)[0]

      # Step 3: Calculate the loss
      loss = loss_module(preds, data_labels)

      # Step 4: Backprop
      # Set all gradients to zero. They would not be overwritten by default, just
      # added to existing values.
      optimizer.zero_grad()
      # Perform backprop
      loss.backward()

      # Step 5: Update the params
      optimizer.step()

      if (i+1) % batch_size == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')

In [10]:
# Instantiate model and place on GPU
model = MNISTGoogLeNet().to(device)

loss_module = nn.CrossEntropyLoss().to(device)

# Input to the optimizer are the parameters of the model: model.parameters()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

train(model, optimizer, minst_train_loader, loss_module, num_epochs, batch_size)

Epoch [1/5], Step [128/469], Loss: 0.0876
Epoch [1/5], Step [256/469], Loss: 0.1107
Epoch [1/5], Step [384/469], Loss: 0.0938
Epoch [2/5], Step [128/469], Loss: 0.0547
Epoch [2/5], Step [256/469], Loss: 0.0637
Epoch [2/5], Step [384/469], Loss: 0.1834
Epoch [3/5], Step [128/469], Loss: 0.0518
Epoch [3/5], Step [256/469], Loss: 0.0198
Epoch [3/5], Step [384/469], Loss: 0.0683
Epoch [4/5], Step [128/469], Loss: 0.0222
Epoch [4/5], Step [256/469], Loss: 0.0182
Epoch [4/5], Step [384/469], Loss: 0.0055
Epoch [5/5], Step [128/469], Loss: 0.0040
Epoch [5/5], Step [256/469], Loss: 0.0151
Epoch [5/5], Step [384/469], Loss: 0.0032


In [11]:
def eval_model(model, data_loader):
  # Set model to eval mode for inference
  model.eval()

  true_preds, num_preds = 0., 0.

  with torch.no_grad(): # Deactivate gradients for inference
    for data_inputs, data_labels in data_loader:
      # Move the data to the same device as the model and loss.
      data_inputs = data_inputs.to(device)
      data_labels = data_labels.to(device)
      # Output is [batchsize, 10]. 10 is because each has a probability for each
      # number's prediction
      preds = model(data_inputs)
      # Take the max over the softmax results aka the probability the picture is
      # a drawing of a number between 0-9. The softmax results are in dim=1.
      # torch.max outputs the (64 max_probabilities, 64 indicies for corresponding softmax value)
      # Index for a softmax value maps to the number it represents, i.e.
      # softmax_class[5] returns the number 5.
      pred = torch.max(preds, dim=1, keepdim=True)[1]
      # View  data_labels as the same size as pred. Move to the CPU and compute
      # the sum of the number of equal/correct predictions.
      true_preds += pred.eq(data_labels.view_as(pred)).cpu().sum()
      # Get the number of predictions by looking at the first index of the shape
      # which is typically m num training examples.
      num_preds += data_labels.shape[0]
    print('Accuracy: {:.4f}'.format(100.0*float(true_preds) / len(data_loader.dataset)))

eval_model(model, mnist_test_loader)

Accuracy: 98.9900
